In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from extract_rule_based_features import ExtractFeatures, detect_bead

In [6]:
def slice_end_to_end(path):
    extractor = ExtractFeatures(path)
    data = extractor.data
    if 'Abnormal' in path:
        data['label'] = 1
        data['dataset_idx'] = path[-12:-10]
    else:
        data['label'] = 0
        data['dataset_idx'] = str(path).split('/')[-1].split('.')[0]

    bead_array = detect_bead(data['LO'], degree=0.17)

    start = bead_array[0,0]
    end = bead_array[-1,1]
    data = data[start:end]

    data['bead_num'] = 0
    data['laser_active'] = 0

    for i in range(len(bead_array)):
        data.loc[bead_array[i][0]:bead_array[i][1],'laser_active'] = 1 # 휴지기간이 아닌 곳을 인덱싱 하기 위함
        data.loc[bead_array[i][0]:bead_array[i][1],'bead_num'] = int(i+1) # 비드 index

    return data


Detected bead num: 54


/tmp/ipykernel_91342/4146634001.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bead_num'] = 0
/tmp/ipykernel_91342/4146634001.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['laser_active'] = 0


In [7]:
from extract_rule_based_features import lo_feature_extract, br_feature_extract, nir_feature_extractor, vis_feature_extractor

In [65]:
from glob import glob
from tqdm import tqdm

raw_dirs = sorted(glob('/workspace/data/230901/양극 테스트_/raw_원천data/*.csv'))

In [103]:
feature_names = ['LO', 'BR', 'NIR', 'VIS']

function_mapper = {'LO': lo_feature_extract, 
                    'BR': br_feature_extract, 
                    'NIR': nir_feature_extractor, 
                    'VIS': vis_feature_extractor}

feature_name = ['lo_height_mean' ,'lo_height_min', 'lo_height_peak', 'lo_height_std', 'lo_fwhm','lo_area',
                'br_peak', 'br_peak_time', 'br_diff_peak',
                'nir_peak', 'nir_reach_time', 'nir_over_area', 'nir_extra_time','nir_extra_area','nir_sum_area',
                'vis_peak', 'vis_reach_time', 'vis_over_area', 'vis_extra_time', 'vis_extra_area'
                ]

final_ruled_list = []

for path in tqdm(raw_dirs):

    file_name = path.split('/')[-1].split('.')[0]

    df = slice_end_to_end(path)

    feature_list = []

    for feature in feature_names:

        values = df[df['bead_num']!=0].groupby('bead_num').apply(lambda x: list(function_mapper[feature](x[feature].to_numpy()).values()))

        # get mean value of each feature for every bead

        # concatenate all features
        feature_dim = len(values[1])
        
        values = np.concatenate(values.to_numpy()).reshape(-1, feature_dim)
        feature_list.append(values)

    pd.DataFrame(np.concatenate(feature_list, axis = 1), columns=feature_name).to_csv(f'/workspace/data/processed/양극/{file_name}_rule_based.csv') 


    rule_mean = np.mean(np.concatenate(feature_list, axis = 1), axis = 0)
    rule_std = np.std(np.concatenate(feature_list, axis = 1), axis = 0)
    final_feature_name = feature_name + list(map(lambda x: x+'_std',feature_name))

    final_ruled = np.concatenate([rule_mean, rule_std], axis = 0)
    
    final_ruled_list.append(final_ruled)
    # final_ruled = pd.DataFrame(np.concatenate([rule_mean, rule_std], axis = 0)).T
    # final_ruled.columns = final_feature_name

  0%|          | 0/39 [00:00<?, ?it/s]/tmp/ipykernel_91342/4146634001.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bead_num'] = 0
/tmp/ipykernel_91342/4146634001.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['laser_active'] = 0
  3%|▎         | 1/39 [00:01<01:04,  1.71s/it]

Detected bead num: 54


/tmp/ipykernel_91342/4146634001.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bead_num'] = 0
/tmp/ipykernel_91342/4146634001.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['laser_active'] = 0
  5%|▌         | 2/39 [00:03<01:05,  1.78s/it]

Detected bead num: 54


/tmp/ipykernel_91342/4146634001.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bead_num'] = 0
/tmp/ipykernel_91342/4146634001.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['laser_active'] = 0
  8%|▊         | 3/39 [00:05<01:00,  1.68s/it]

...

/tmp/ipykernel_91342/4146634001.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bead_num'] = 0
/tmp/ipykernel_91342/4146634001.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['laser_active'] = 0
 97%|█████████▋| 38/39 [01:06<00:01,  1.86s/it]

Detected bead num: 54


/tmp/ipykernel_91342/4146634001.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bead_num'] = 0
/tmp/ipykernel_91342/4146634001.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['laser_active'] = 0
100%|██████████| 39/39 [01:07<00:00,  1.74s/it]

Detected bead num: 54


ValueError: Shape of passed values is (40, 1), indices imply (40, 40)

In [124]:
df_LG = pd.read_excel('/workspace/data/230901/양극 인장강도 매칭.xlsx', sheet_name='data').loc[:38] # 원본 그대로 읽으면 불필요한 아래 시각화 정보까지 읽게 됨
final_feature_name = list(map(lambda x: x+'_mean',feature_name)) + list(map(lambda x: x+'_std',feature_name)) + ['실인장']
final_ruled_list = np.concatenate((np.array(final_ruled_list),df_LG['실인장'].to_numpy().reshape(-1,1)), axis = 1)

pd.DataFrame(final_ruled_list, columns = final_feature_name).to_csv('/workspace/data/processed/양극/rule_based_final.csv') 

## 음극

In [128]:
raw_dirs = sorted(glob('/workspace/data/230901/3파장모니터링 인장평가_음극_비드구분raw/*.csv'))

feature_names = ['LO', 'BR', 'NIR', 'VIS']

function_mapper = {'LO': lo_feature_extract, 
                    'BR': br_feature_extract, 
                    'NIR': nir_feature_extractor, 
                    'VIS': vis_feature_extractor}

feature_name = ['lo_height_mean' ,'lo_height_min', 'lo_height_peak', 'lo_height_std', 'lo_fwhm','lo_area',
                'br_peak', 'br_peak_time', 'br_diff_peak',
                'nir_peak', 'nir_reach_time', 'nir_over_area', 'nir_extra_time','nir_extra_area','nir_sum_area',
                'vis_peak', 'vis_reach_time', 'vis_over_area', 'vis_extra_time', 'vis_extra_area'
                ]


final_ruled_list = []

for path in tqdm(raw_dirs):

    file_name = path.split('/')[-1].split('.')[0]

    df = pd.read_csv(path)

    feature_list = []

    for feature in feature_names:

        values = df.groupby('Bead_No').apply(lambda x: list(function_mapper[feature](x[feature].to_numpy()).values()))

        # get mean value of each feature for every bead

        # concatenate all features
        feature_dim = len(values[1])
        
        values = np.concatenate(values.to_numpy()).reshape(-1, feature_dim)
        feature_list.append(values)

    pd.DataFrame(np.concatenate(feature_list, axis = 1), columns=feature_name).to_csv(f'/workspace/data/processed/음극/{file_name}_rule_based.csv') 


    rule_mean = np.mean(np.concatenate(feature_list, axis = 1), axis = 0)
    rule_std = np.std(np.concatenate(feature_list, axis = 1), axis = 0)
    final_feature_name = feature_name + list(map(lambda x: x+'_std',feature_name))

    final_ruled = np.concatenate([rule_mean, rule_std], axis = 0)
    
    final_ruled_list.append(final_ruled)

100%|██████████| 15/15 [00:00<00:00, 23.21it/s]


In [129]:
df_LG = pd.read_excel('/workspace/data/230901/음극 인장강도 매칭.xlsx', sheet_name='data').loc[:14] # 원본 그대로 읽으면 불필요한 아래 시각화 정보까지 읽게 됨
final_feature_name = list(map(lambda x: x+'_mean',feature_name)) + list(map(lambda x: x+'_std',feature_name)) + ['인장강도']
final_ruled_list = np.concatenate((np.array(final_ruled_list),df_LG['인장강도'].to_numpy().reshape(-1,1)), axis = 1)

pd.DataFrame(final_ruled_list, columns = final_feature_name).to_csv('/workspace/data/processed/음극/rule_based_final.csv') 